# OOP Inheritance

*📚 Computer Programming II · 👨‍🏫 Dr. Arif Solmaz*


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⏱️ OTURUM ZAMANLAYICI — Bu hücreyi ilk çalıştırın!
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import time as _time, datetime as _dt, json as _json, os as _os
from IPython.display import display, HTML as _HTML

# ── Persistent tracker file (survives kernel restarts in same runtime) ──
_TRACKER_FILE = _os.path.join(
    '/content' if _os.path.isdir('/content') else '/tmp',
    '.cp2_session_Week_11.json'
)

def _load_tracker():
    """Load previous sessions from file."""
    try:
        with open(_TRACKER_FILE, 'r') as f:
            return _json.load(f)
    except (FileNotFoundError, _json.JSONDecodeError, ValueError):
        return {'sessions': [], 'total_heartbeats': 0}

def _save_tracker(data):
    """Persist tracker data to file."""
    try:
        with open(_TRACKER_FILE, 'w') as f:
            _json.dump(data, f)
    except OSError:
        pass

# ── Load any previous session data ──
_tracker = _load_tracker()
_prev_sessions = len(_tracker['sessions'])
_prev_hb = _tracker['total_heartbeats']

# ── Start new session ──
_SESSION_START = _time.time()
_SESSION_START_STR = _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
_HEARTBEATS = [_time.time()]
_CELLS_RUN = [0]
IDLE_THRESHOLD = 300   # 5 min

# Record this session start
_tracker['sessions'].append({
    'start': _SESSION_START_STR,
    'start_ts': _SESSION_START,
    'heartbeats': [_SESSION_START]
})
_save_tracker(_tracker)

def _heartbeat_hook(*args, **kwargs):
    """Record each cell execution — in-memory + file."""
    now = _time.time()
    _HEARTBEATS.append(now)
    _CELLS_RUN[0] += 1
    # Persist every 5 cells to avoid excessive disk IO
    if _CELLS_RUN[0] % 5 == 0:
        try:
            _t = _load_tracker()
            if _t['sessions']:
                _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
                _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
                _t['total_heartbeats'] = _prev_hb + len(_HEARTBEATS)
                _save_tracker(_t)
        except Exception:
            pass

def _calc_active_time(heartbeats=None):
    """Calculate active time from heartbeat list, capping idle gaps."""
    hb = heartbeats or _HEARTBEATS
    if len(hb) < 2: return 0
    active = 0
    for i in range(1, len(hb)):
        gap = hb[i] - hb[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _calc_total_active_time():
    """Calculate total across ALL sessions (multi-day support)."""
    total = 0
    try:
        _t = _load_tracker()
        for sess in _t['sessions'][:-1]:  # all previous sessions
            hb = sess.get('heartbeats', [])
            if len(hb) >= 2:
                for i in range(1, len(hb)):
                    gap = hb[i] - hb[i-1]
                    total += gap if gap <= IDLE_THRESHOLD else 30
    except Exception:
        pass
    # Add current session
    total += _calc_active_time()
    return int(total)

try:
    _ip = get_ipython()
    _ip.events.register('pre_run_cell', _heartbeat_hook)
except Exception: pass

# ── Display ──
_multi = ''
if _prev_sessions > 0:
    _multi = f'<br><span style="font-size:12px;opacity:.75">📂 {_prev_sessions} önceki oturum bulundu — çoklu oturum takibi aktif.</span>'
display(_HTML(f"""<div style='background:linear-gradient(135deg,#667eea,#764ba2);padding:14px 20px;border-radius:10px;color:white;font-family:system-ui;margin:4px 0'>
<b>⏱️ Oturum Başladı</b> — {_SESSION_START_STR}<br>
<span style='font-size:13px;opacity:.85'>Hücre aktiviteniz takip ediliyor. Bitince en alttaki Submit hücresini çalıştırın.</span>{_multi}</div>"""))
print(f'✅ Zamanlayıcı aktif. Idle eşiği: {IDLE_THRESHOLD//60} dk')
if _prev_sessions > 0:
    print(f'📂 Önceki oturumlar: {_prev_sessions} | Toplam heartbeat: {_prev_hb}')

## 🎯 Learning Objectives

- Understand inheritance and its benefits
- Create subclasses that extend parent classes
- Override methods in child classes
- Use the super() function effectively
- Implement multiple inheritance
- Create abstract base classes
- Apply polymorphism in your code
- Design class hierarchies for engineering applications

---
## Part 1: Introduction to Inheritance


Inheritance is one of the four pillars of Object-Oriented Programming. It allows a class (child/subclass) to inherit attributes and methods from another class (parent/superclass). This promotes code reuse and establishes "is-a" relationships between classes.

| Term | Also Known As | Description |
| --- | --- | --- |
| **Parent Class** | Base Class, Superclass | The class being inherited from |
| **Child Class** | Derived Class, Subclass | The class that inherits |
| **Inheritance** | Derivation | Mechanism to reuse code from parent |

### Without Inheritance - Code Duplication

**Figure 1.1: Code Duplication Problem**

In [ ]:
# Without inheritance - code duplication
class Dog:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def eat(self):
        return f"{self.name} is eating"
    
    def sleep(self):
        return f"{self.name} is sleeping"
    
    def bark(self):
        return f"{self.name} says Woof!"

class Cat:
    def __init__(self, name, age):  # Duplicated!
        self.name = name
        self.age = age
    
    def eat(self):  # Duplicated!
        return f"{self.name} is eating"
    
    def sleep(self):  # Duplicated!
        return f"{self.name} is sleeping"
    
    def meow(self):
        return f"{self.name} says Meow!"

print("Problem: __init__, eat(), and sleep() are duplicated!")
print("If we need to change eat(), we must change it in BOTH classes.")

### With Inheritance - No Duplication

**Figure 1.2: Inheritance Solution**

In [ ]:
# With inheritance - no duplication
class Animal:
    """Base class for all animals."""
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def eat(self):
        return f"{self.name} is eating"
    
    def sleep(self):
        return f"{self.name} is sleeping"

class Dog(Animal):  # Dog inherits from Animal
    """Dog class inheriting from Animal."""
    def bark(self):
        return f"{self.name} says Woof!"

class Cat(Animal):  # Cat inherits from Animal
    """Cat class inheriting from Animal."""
    def meow(self):
        return f"{self.name} says Meow!"

# Create instances
dog = Dog("Buddy", 3)
cat = Cat("Whiskers", 2)

# Inherited methods work
print(f"Dog: {dog.name}, {dog.age} years old")
print(dog.eat())     # Inherited from Animal
print(dog.sleep())   # Inherited from Animal
print(dog.bark())    # Defined in Dog
print()
print(f"Cat: {cat.name}, {cat.age} years old")
print(cat.eat())     # Inherited from Animal
print(cat.meow())    # Defined in Cat

### Checking Inheritance Relationships

**Figure 1.3: Inheritance Checking**

In [ ]:
# Checking inheritance relationships
class Animal:
    pass

class Dog(Animal):
    pass

class Cat(Animal):
    pass

dog = Dog()
cat = Cat()

print("=== isinstance() checks ===")
print(f"isinstance(dog, Dog): {isinstance(dog, Dog)}")
print(f"isinstance(dog, Animal): {isinstance(dog, Animal)}")  # True!
print(f"isinstance(dog, Cat): {isinstance(dog, Cat)}")
print(f"isinstance(dog, object): {isinstance(dog, object)}")  # True!
print()

print("=== issubclass() checks ===")
print(f"issubclass(Dog, Animal): {issubclass(Dog, Animal)}")
print(f"issubclass(Cat, Animal): {issubclass(Cat, Animal)}")
print(f"issubclass(Dog, Cat): {issubclass(Dog, Cat)}")
print(f"issubclass(Animal, object): {issubclass(Animal, object)}")
print()

print("=== Class hierarchy ===")
print(f"Dog.__bases__: {Dog.__bases__}")
print(f"Animal.__bases__: {Animal.__bases__}")

> 💡 **Note:** Remember: Inheritance represents an "is-a" relationship. A Dog "is-a" Animal. Use inheritance when this relationship makes sense conceptually.

---
## Part 2: The super() Function


The `super()` function returns a proxy object that delegates method calls to the parent class. It's essential for calling parent class methods, especially `__init__`, from the child class.

### Why We Need super()

**Figure 2.1: The Problem Without super()**

In [ ]:
# Problem: Child class overrides __init__ completely
class Animal:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        print(f"Animal.__init__ called for {name}")

class Dog(Animal):
    def __init__(self, name, age, breed):
        # This REPLACES Animal.__init__, not extends it!
        self.breed = breed
        print(f"Dog.__init__ called for {name}")

try:
    dog = Dog("Buddy", 3, "Golden Retriever")
    print(f"Dog's name: {dog.name}")  # This will fail!
except AttributeError as e:
    print(f"Error: {e}")
    print("Animal.__init__ was never called, so name/age weren't set!")

### Using super() Correctly

**Figure 2.2: Correct Use of super()**

In [ ]:
# Correct: Use super() to call parent __init__
class Animal:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        print(f"Animal.__init__ called for {name}")

class Dog(Animal):
    def __init__(self, name, age, breed):
        # Call parent's __init__ first
        super().__init__(name, age)
        # Then add Dog-specific attributes
        self.breed = breed
        print(f"Dog.__init__ called for {name}")

dog = Dog("Buddy", 3, "Golden Retriever")
print()
print(f"Name: {dog.name}")      # From Animal
print(f"Age: {dog.age}")        # From Animal
print(f"Breed: {dog.breed}")    # From Dog

### super() with Other Methods

**Figure 2.3: super() in Methods**

In [ ]:
# Using super() to extend (not replace) methods
class Vehicle:
    def __init__(self, brand, model):
        self.brand = brand
        self.model = model
    
    def describe(self):
        return f"{self.brand} {self.model}"
    
    def start(self):
        return "Vehicle starting..."

class Car(Vehicle):
    def __init__(self, brand, model, num_doors):
        super().__init__(brand, model)
        self.num_doors = num_doors
    
    def describe(self):
        # Extend parent's describe
        base = super().describe()
        return f"{base} ({self.num_doors} doors)"
    
    def start(self):
        # Extend parent's start
        base = super().start()
        return f"{base} Engine ignited!"

car = Car("Toyota", "Camry", 4)
print(car.describe())
print(car.start())

#### Engineering Example: Sensor Hierarchy

**Figure 2.4: Sensor Class Hierarchy**

In [ ]:
# Engineering: Sensor class hierarchy
class Sensor:
    """Base sensor class."""
    sensor_count = 0
    
    def __init__(self, sensor_id, unit):
        self.sensor_id = sensor_id
        self.unit = unit
        self.value = None
        Sensor.sensor_count += 1
    
    def read(self, value):
        self.value = value
        return self.value
    
    def display(self):
        if self.value is None:
            return f"{self.sensor_id}: No reading"
        return f"{self.sensor_id}: {self.value} {self.unit}"

class TemperatureSensor(Sensor):
    """Temperature sensor with calibration."""
    
    def __init__(self, sensor_id, calibration_offset=0):
        super().__init__(sensor_id, "°C")
        self.calibration_offset = calibration_offset
    
    def read(self, value):
        # Apply calibration before storing
        calibrated = value + self.calibration_offset
        return super().read(calibrated)

class PressureSensor(Sensor):
    """Pressure sensor with altitude compensation."""
    
    def __init__(self, sensor_id, altitude=0):
        super().__init__(sensor_id, "hPa")
        self.altitude = altitude
    
    def read(self, value):
        # Apply altitude compensation
        compensated = value + (self.altitude * 0.12)
        return super().read(round(compensated, 2))

# Create sensors
temp = TemperatureSensor("TEMP_001", calibration_offset=0.5)
pressure = PressureSensor("PRES_001", altitude=100)

temp.read(25.0)
pressure.read(1013.25)

print(temp.display())
print(pressure.display())
print(f"Total sensors: {Sensor.sensor_count}")

---
## Part 3: Method Overriding


Method overriding occurs when a child class provides its own implementation of a method that already exists in the parent class. The child's version replaces (or extends) the parent's version when called on child objects.

### Basic Method Overriding

**Figure 3.1: Overriding Methods**

In [ ]:
# Method overriding
class Animal:
    def __init__(self, name):
        self.name = name
    
    def speak(self):
        return "Some generic animal sound"
    
    def move(self):
        return f"{self.name} is moving"

class Dog(Animal):
    def speak(self):  # Override
        return f"{self.name} says Woof!"

class Cat(Animal):
    def speak(self):  # Override
        return f"{self.name} says Meow!"

class Fish(Animal):
    def speak(self):  # Override
        return f"{self.name} makes no sound (it's a fish!)"
    
    def move(self):  # Override
        return f"{self.name} is swimming"

# Create animals
animals = [
    Dog("Buddy"),
    Cat("Whiskers"),
    Fish("Nemo")
]

for animal in animals:
    print(f"{animal.speak()}")
    print(f"  {animal.move()}")
    print()

### Overriding __str__ and __repr__

**Figure 3.2: Overriding Special Methods**

In [ ]:
# Overriding special methods
class Person:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"Person: {self.name}"
    
    def __repr__(self):
        return f"Person('{self.name}')"

class Employee(Person):
    def __init__(self, name, employee_id):
        super().__init__(name)
        self.employee_id = employee_id
    
    def __str__(self):  # Override
        return f"Employee: {self.name} (ID: {self.employee_id})"
    
    def __repr__(self):  # Override
        return f"Employee('{self.name}', '{self.employee_id}')"

class Manager(Employee):
    def __init__(self, name, employee_id, department):
        super().__init__(name, employee_id)
        self.department = department
    
    def __str__(self):  # Override
        return f"Manager: {self.name} - {self.department} (ID: {self.employee_id})"

person = Person("Ali")
employee = Employee("Ayşe", "E001")
manager = Manager("Mehmet", "M001", "Engineering")

print(person)
print(employee)
print(manager)

#### Engineering Example: Motor Types

**Figure 3.3: Different Motor Types**

In [ ]:
# Engineering: Different motor types with overriding
class Motor:
    def __init__(self, motor_id, max_rpm):
        self.motor_id = motor_id
        self.max_rpm = max_rpm
        self.current_rpm = 0
        self.is_running = False
    
    def start(self):
        self.is_running = True
        return f"{self.motor_id}: Started"
    
    def stop(self):
        self.is_running = False
        self.current_rpm = 0
        return f"{self.motor_id}: Stopped"
    
    def set_speed(self, rpm):
        if not self.is_running:
            return f"{self.motor_id}: Must start first!"
        self.current_rpm = min(rpm, self.max_rpm)
        return f"{self.motor_id}: Speed set to {self.current_rpm} RPM"

class StepperMotor(Motor):
    def __init__(self, motor_id, max_rpm, steps_per_rev):
        super().__init__(motor_id, max_rpm)
        self.steps_per_rev = steps_per_rev
        self.current_step = 0
    
    def start(self):  # Override - add homing
        self.current_step = 0
        result = super().start()
        return f"{result} (Homed to step 0)"
    
    def move_steps(self, steps):
        if not self.is_running:
            return f"{self.motor_id}: Must start first!"
        self.current_step += steps
        return f"{self.motor_id}: Moved to step {self.current_step}"

class ServoMotor(Motor):
    def __init__(self, motor_id, min_angle=0, max_angle=180):
        super().__init__(motor_id, 60)  # Servos have low RPM
        self.min_angle = min_angle
        self.max_angle = max_angle
        self.current_angle = 0
    
    def set_angle(self, angle):
        if not self.is_running:
            return f"{self.motor_id}: Must start first!"
        self.current_angle = max(self.min_angle, min(angle, self.max_angle))
        return f"{self.motor_id}: Angle set to {self.current_angle}°"

# Test motors
dc = Motor("DC_001", 3000)
stepper = StepperMotor("STEP_001", 1000, 200)
servo = ServoMotor("SERVO_001")

print(dc.start())
print(dc.set_speed(1500))
print()
print(stepper.start())
print(stepper.move_steps(100))
print()
print(servo.start())
print(servo.set_angle(90))

---
## Part 4: Polymorphism


**Polymorphism** means "many forms" - the same interface can work with objects of different types. This allows writing flexible code that works with any object implementing a common interface, regardless of its specific class.

### Polymorphism with Shapes

**Figure 4.1: Polymorphic Shapes**

In [ ]:
# Polymorphism with shapes
class Shape:
    def area(self):
        raise NotImplementedError("Subclass must implement")
    
    def perimeter(self):
        raise NotImplementedError("Subclass must implement")

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def area(self):
        return self.width * self.height
    
    def perimeter(self):
        return 2 * (self.width + self.height)

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    
    def area(self):
        return 3.14159 * self.radius ** 2
    
    def perimeter(self):
        return 2 * 3.14159 * self.radius

class Triangle(Shape):
    def __init__(self, a, b, c):
        self.a, self.b, self.c = a, b, c
    
    def area(self):
        s = self.perimeter() / 2
        return (s * (s-self.a) * (s-self.b) * (s-self.c)) ** 0.5
    
    def perimeter(self):
        return self.a + self.b + self.c

# Polymorphic function - works with ANY Shape
def print_shape_info(shape):
    name = shape.__class__.__name__
    print(f"{name}: area={shape.area():.2f}, perimeter={shape.perimeter():.2f}")

# Create different shapes
shapes = [
    Rectangle(10, 5),
    Circle(7),
    Triangle(3, 4, 5)
]

print("=== All shapes through same interface ===")
for shape in shapes:
    print_shape_info(shape)

# Calculate total area
total = sum(s.area() for s in shapes)
print(f"\nTotal area: {total:.2f}")

### Duck Typing

**Figure 4.2: Duck Typing**

In [ ]:
# Duck typing: "If it walks like a duck and quacks like a duck..."
class Duck:
    def quack(self):
        return "Quack!"
    
    def fly(self):
        return "Flying with wings"

class RobotDuck:
    def quack(self):
        return "Electronic quack!"
    
    def fly(self):
        return "Flying with propellers"

class Person:
    def quack(self):
        return "I'm pretending to quack!"
    
    def fly(self):
        return "I can't fly, but I can run!"

# Function doesn't care about type, only interface
def make_it_do_duck_things(duck_like_thing):
    print(f"  Quack: {duck_like_thing.quack()}")
    print(f"  Fly: {duck_like_thing.fly()}")

print("Real Duck:")
make_it_do_duck_things(Duck())

print("\nRobot Duck:")
make_it_do_duck_things(RobotDuck())

print("\nPerson (pretending):")
make_it_do_duck_things(Person())

#### Engineering Example: Data Processors

**Figure 4.3: Polymorphic Data Processors**

In [ ]:
# Engineering: Polymorphic data processors
class DataProcessor:
    """Base class for data processors."""
    def process(self, data):
        raise NotImplementedError

class AverageProcessor(DataProcessor):
    def process(self, data):
        return sum(data) / len(data) if data else None

class MaxProcessor(DataProcessor):
    def process(self, data):
        return max(data) if data else None

class MinMaxProcessor(DataProcessor):
    def process(self, data):
        if not data:
            return None
        return {'min': min(data), 'max': max(data)}

class FilteredAverageProcessor(DataProcessor):
    def __init__(self, min_val=None, max_val=None):
        self.min_val = min_val
        self.max_val = max_val
    
    def process(self, data):
        filtered = data
        if self.min_val is not None:
            filtered = [x for x in filtered if x >= self.min_val]
        if self.max_val is not None:
            filtered = [x for x in filtered if x <= self.max_val]
        return sum(filtered) / len(filtered) if filtered else None

# Sensor data
readings = [23.5, 24.1, 100.0, 24.8, 25.2, -5.0, 24.5]
print(f"Data: {readings}")
print()

# Process with different processors (polymorphism!)
processors = [
    ("Average", AverageProcessor()),
    ("Max", MaxProcessor()),
    ("MinMax", MinMaxProcessor()),
    ("Filtered Avg (20-30)", FilteredAverageProcessor(20, 30))
]

for name, processor in processors:
    result = processor.process(readings)
    print(f"{name}: {result}")

---
## Part 5: Multiple Inheritance


Python supports multiple inheritance, where a class can inherit from more than one parent class. This is powerful but requires understanding of the Method Resolution Order (MRO).

### Basic Multiple Inheritance

**Figure 5.1: Multiple Inheritance**

In [ ]:
# Multiple inheritance
class Flyable:
    def fly(self):
        return f"{self.name} is flying"

class Swimmable:
    def swim(self):
        return f"{self.name} is swimming"

class Walkable:
    def walk(self):
        return f"{self.name} is walking"

class Duck(Flyable, Swimmable, Walkable):
    def __init__(self, name):
        self.name = name

class Penguin(Swimmable, Walkable):
    def __init__(self, name):
        self.name = name

class Eagle(Flyable, Walkable):
    def __init__(self, name):
        self.name = name

duck = Duck("Donald")
penguin = Penguin("Happy Feet")
eagle = Eagle("Eddie")

print(f"Duck: {duck.fly()}, {duck.swim()}, {duck.walk()}")
print(f"Penguin: {penguin.swim()}, {penguin.walk()}")
print(f"Eagle: {eagle.fly()}, {eagle.walk()}")

### Method Resolution Order (MRO)

**Figure 5.2: Understanding MRO**

In [ ]:
# Method Resolution Order (MRO)
class A:
    def greet(self):
        return "Hello from A"

class B(A):
    def greet(self):
        return "Hello from B"

class C(A):
    def greet(self):
        return "Hello from C"

class D(B, C):  # Diamond inheritance
    pass

class E(C, B):  # Different order
    pass

# Check MRO
print("D's MRO (inherits B, C):")
for cls in D.__mro__:
    print(f"  {cls.__name__}")

print()
print("E's MRO (inherits C, B):")
for cls in E.__mro__:
    print(f"  {cls.__name__}")

print()
d = D()
e = E()
print(f"D.greet(): {d.greet()}")  # Uses B's greet (first in MRO)
print(f"E.greet(): {e.greet()}")  # Uses C's greet (first in MRO)

### Mixins Pattern

**Figure 5.3: Mixin Classes**

In [ ]:
# Mixins: Small classes that provide specific functionality
class LoggableMixin:
    """Mixin that adds logging capability."""
    def log(self, message):
        print(f"[{self.__class__.__name__}] {message}")

class SerializableMixin:
    """Mixin that adds serialization capability."""
    def to_dict(self):
        return {k: v for k, v in self.__dict__.items() if not k.startswith('_')}
    
    def to_string(self):
        items = [f"{k}={v}" for k, v in self.to_dict().items()]
        return f"{self.__class__.__name__}({', '.join(items)})"

class ValidatableMixin:
    """Mixin that adds validation capability."""
    def validate(self):
        for attr, value in self.__dict__.items():
            if value is None:
                return False, f"{attr} is None"
        return True, "Valid"

class Sensor(LoggableMixin, SerializableMixin, ValidatableMixin):
    """Sensor class with logging, serialization, and validation."""
    def __init__(self, sensor_id, sensor_type):
        self.sensor_id = sensor_id
        self.sensor_type = sensor_type
        self.value = None
    
    def read(self, value):
        self.value = value
        self.log(f"Read value: {value}")

# Test the sensor with mixins
sensor = Sensor("TEMP_001", "temperature")
sensor.read(25.5)

print(f"As dict: {sensor.to_dict()}")
print(f"As string: {sensor.to_string()}")

valid, msg = sensor.validate()
print(f"Valid: {valid} - {msg}")

> 💡 **Note:** Use multiple inheritance carefully! Prefer composition over complex inheritance hierarchies. Mixins are a good pattern - they add specific capabilities without complex relationships.

---
## Part 6: Abstract Base Classes


Abstract Base Classes (ABCs) define interfaces that child classes must implement. They cannot be instantiated directly and enforce a contract that subclasses must follow.

### Creating Abstract Classes

**Figure 6.1: Abstract Base Classes**

In [ ]:
# Abstract Base Classes
from abc import ABC, abstractmethod

class Shape(ABC):
    """Abstract base class for shapes."""
    
    @abstractmethod
    def area(self):
        """Calculate area - must be implemented by subclasses."""
        pass
    
    @abstractmethod
    def perimeter(self):
        """Calculate perimeter - must be implemented by subclasses."""
        pass
    
    def describe(self):
        """Concrete method - inherited by subclasses."""
        return f"{self.__class__.__name__}: area={self.area():.2f}"

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def area(self):  # Must implement
        return self.width * self.height
    
    def perimeter(self):  # Must implement
        return 2 * (self.width + self.height)

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    
    def area(self):  # Must implement
        return 3.14159 * self.radius ** 2
    
    def perimeter(self):  # Must implement
        return 2 * 3.14159 * self.radius

# Cannot instantiate abstract class
try:
    shape = Shape()
except TypeError as e:
    print(f"Cannot create Shape: {e}")
    print()

# Can instantiate concrete classes
rect = Rectangle(10, 5)
circle = Circle(7)

print(rect.describe())
print(circle.describe())

### Abstract Properties

**Figure 6.2: Abstract Properties**

In [ ]:
# Abstract properties
from abc import ABC, abstractmethod

class Vehicle(ABC):
    """Abstract vehicle class."""
    
    @property
    @abstractmethod
    def max_speed(self):
        """Maximum speed - must be implemented."""
        pass
    
    @property
    @abstractmethod
    def fuel_type(self):
        """Fuel type - must be implemented."""
        pass
    
    @abstractmethod
    def start(self):
        pass

class Car(Vehicle):
    def __init__(self, brand):
        self.brand = brand
        self._max_speed = 200
    
    @property
    def max_speed(self):
        return self._max_speed
    
    @property
    def fuel_type(self):
        return "gasoline"
    
    def start(self):
        return f"{self.brand} car starting..."

class ElectricCar(Vehicle):
    def __init__(self, brand):
        self.brand = brand
        self._max_speed = 250
    
    @property
    def max_speed(self):
        return self._max_speed
    
    @property
    def fuel_type(self):
        return "electric"
    
    def start(self):
        return f"{self.brand} electric car starting silently..."

car = Car("Toyota")
ev = ElectricCar("Tesla")

print(f"{car.brand}: {car.max_speed} km/h, {car.fuel_type}")
print(f"  {car.start()}")
print()
print(f"{ev.brand}: {ev.max_speed} km/h, {ev.fuel_type}")
print(f"  {ev.start()}")

#### Engineering Example: Sensor Interface

**Figure 6.3: Abstract Sensor Interface**

In [ ]:
# Engineering: Abstract sensor interface
from abc import ABC, abstractmethod

class SensorInterface(ABC):
    """Abstract interface for all sensors."""
    
    @property
    @abstractmethod
    def sensor_id(self):
        pass
    
    @property
    @abstractmethod
    def unit(self):
        pass
    
    @abstractmethod
    def read(self):
        """Read current value from sensor."""
        pass
    
    @abstractmethod
    def calibrate(self, reference_value):
        """Calibrate sensor with reference value."""
        pass

class TemperatureSensor(SensorInterface):
    def __init__(self, sid):
        self._sensor_id = sid
        self._value = 0
        self._offset = 0
    
    @property
    def sensor_id(self):
        return self._sensor_id
    
    @property
    def unit(self):
        return "°C"
    
    def read(self):
        # Simulate reading
        return self._value + self._offset
    
    def calibrate(self, reference_value):
        self._offset = reference_value - self._value
        return f"Calibrated with offset {self._offset}"
    
    def set_raw(self, value):
        self._value = value

class HumiditySensor(SensorInterface):
    def __init__(self, sid):
        self._sensor_id = sid
        self._value = 0
        self._scale = 1.0
    
    @property
    def sensor_id(self):
        return self._sensor_id
    
    @property
    def unit(self):
        return "%"
    
    def read(self):
        return self._value * self._scale
    
    def calibrate(self, reference_value):
        if self._value != 0:
            self._scale = reference_value / self._value
        return f"Calibrated with scale {self._scale:.2f}"
    
    def set_raw(self, value):
        self._value = value

# Use the sensors
temp = TemperatureSensor("TEMP_001")
temp.set_raw(24.5)
print(f"{temp.sensor_id}: {temp.read()} {temp.unit}")
print(temp.calibrate(25.0))
print(f"After calibration: {temp.read()} {temp.unit}")

print()

humid = HumiditySensor("HUMID_001")
humid.set_raw(45)
print(f"{humid.sensor_id}: {humid.read()} {humid.unit}")
print(humid.calibrate(50))
print(f"After calibration: {humid.read():.1f} {humid.unit}")

---
## Part 7: Inheritance Best Practices


Inheritance is powerful but can lead to complex, hard-to-maintain code if misused. Follow these best practices for cleaner, more maintainable object-oriented designs.

| Principle | Description |
| --- | --- |
| **Liskov Substitution** | Subclass objects should be usable wherever parent class objects are expected |
| **Favor Composition** | Prefer "has-a" over "is-a" relationships when possible |
| **Keep It Shallow** | Avoid deep inheritance hierarchies (max 2-3 levels) |
| **Single Responsibility** | Each class should have one reason to change |

### Liskov Substitution Principle

**Figure 7.1: Liskov Substitution**

In [ ]:
# Liskov Substitution Principle (LSP)
# Bad example: Square inheriting from Rectangle breaks LSP
class Rectangle:
    def __init__(self, width, height):
        self._width = width
        self._height = height
    
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, value):
        self._width = value
    
    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, value):
        self._height = value
    
    def area(self):
        return self._width * self._height

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)
    
    @Rectangle.width.setter
    def width(self, value):
        self._width = value
        self._height = value  # Force square
    
    @Rectangle.height.setter
    def height(self, value):
        self._width = value
        self._height = value  # Force square

# This function expects Rectangle behavior
def process_rectangle(rect):
    rect.width = 5
    rect.height = 10
    expected = 50  # 5 * 10
    actual = rect.area()
    return expected == actual

rect = Rectangle(1, 1)
square = Square(1)

print(f"Rectangle passes: {process_rectangle(rect)}")
print(f"Square passes: {process_rectangle(square)}")  # Fails LSP!
print()
print("Square breaks LSP because setting width/height separately")
print("doesn't work as expected for a Rectangle interface.")

### Composition Over Inheritance

**Figure 7.2: Composition Example**

In [ ]:
# Composition over inheritance
# Instead of: Robot inherits from Arm, Sensor, Motor (messy!)
# Use: Robot HAS-A Arm, HAS-A Sensor, HAS-A Motor

class Arm:
    def __init__(self, reach):
        self.reach = reach
    
    def extend(self):
        return f"Arm extended {self.reach}cm"
    
    def retract(self):
        return "Arm retracted"

class Sensor:
    def __init__(self, sensor_type):
        self.sensor_type = sensor_type
    
    def read(self):
        return f"{self.sensor_type} reading: OK"

class Motor:
    def __init__(self, power):
        self.power = power
    
    def move(self, direction):
        return f"Moving {direction} at {self.power}W"

class Robot:
    """Robot composed of multiple components."""
    
    def __init__(self, name):
        self.name = name
        # Composition: Robot HAS-A these components
        self.arm = Arm(reach=50)
        self.sensor = Sensor("proximity")
        self.motor = Motor(power=100)
    
    def pick_object(self):
        results = []
        results.append(self.sensor.read())
        results.append(self.arm.extend())
        results.append(self.arm.retract())
        return results
    
    def move_to(self, location):
        return self.motor.move(location)

# Use the robot
robot = Robot("Helper-1")
print(f"Robot: {robot.name}")
print(f"Moving: {robot.move_to('forward')}")
print("Picking object:")
for action in robot.pick_object():
    print(f"  - {action}")

> 💡 **Note:** **When to use inheritance:** Use inheritance for "is-a" relationships where the child truly IS a specialized version of the parent. Use composition for "has-a" relationships where an object contains other objects.

---
## Part 8: Practical Applications


Let's apply inheritance concepts to real engineering problems.

### Control System Hierarchy

**Figure 8.1: Control System**

In [ ]:
# Engineering: Control system hierarchy
from abc import ABC, abstractmethod

class Controller(ABC):
    """Abstract base controller."""
    
    def __init__(self, name, setpoint):
        self.name = name
        self.setpoint = setpoint
        self.output = 0
    
    @abstractmethod
    def compute(self, current_value):
        """Compute control output."""
        pass
    
    def status(self):
        return f"{self.name}: setpoint={self.setpoint}, output={self.output:.2f}"

class OnOffController(Controller):
    """Simple on/off controller."""
    
    def __init__(self, name, setpoint, hysteresis=1):
        super().__init__(name, setpoint)
        self.hysteresis = hysteresis
    
    def compute(self, current_value):
        if current_value < self.setpoint - self.hysteresis:
            self.output = 100  # Full on
        elif current_value > self.setpoint + self.hysteresis:
            self.output = 0    # Full off
        return self.output

class PController(Controller):
    """Proportional controller."""
    
    def __init__(self, name, setpoint, kp=1.0):
        super().__init__(name, setpoint)
        self.kp = kp
    
    def compute(self, current_value):
        error = self.setpoint - current_value
        self.output = self.kp * error
        return self.output

class PIController(PController):
    """Proportional-Integral controller."""
    
    def __init__(self, name, setpoint, kp=1.0, ki=0.1):
        super().__init__(name, setpoint, kp)
        self.ki = ki
        self.integral = 0
    
    def compute(self, current_value):
        error = self.setpoint - current_value
        self.integral += error
        self.output = self.kp * error + self.ki * self.integral
        return self.output

# Test controllers
controllers = [
    OnOffController("Heater", 25),
    PController("Motor Speed", 1000, kp=0.5),
    PIController("Temperature", 50, kp=2.0, ki=0.1)
]

current_values = [23, 800, 45]

print("Controller Outputs:")
print("-" * 50)
for ctrl, value in zip(controllers, current_values):
    output = ctrl.compute(value)
    print(f"{ctrl.name}: input={value}, output={output:.2f}")

### Data Logger System

**Figure 8.2: Data Logger System**

In [ ]:
# Engineering: Data logger system
from abc import ABC, abstractmethod

class DataLogger(ABC):
    """Abstract data logger."""
    
    def __init__(self, name):
        self.name = name
        self.entries = []
    
    @abstractmethod
    def format_entry(self, value):
        """Format a log entry."""
        pass
    
    def log(self, value):
        entry = self.format_entry(value)
        self.entries.append(entry)
        return entry
    
    def get_all(self):
        return self.entries.copy()

class SimpleLogger(DataLogger):
    """Simple text logger."""
    
    def format_entry(self, value):
        return f"{len(self.entries)}: {value}"

class TimestampLogger(DataLogger):
    """Logger with timestamps."""
    
    def __init__(self, name):
        super().__init__(name)
        self.time = 0
    
    def format_entry(self, value):
        entry = f"[t={self.time:04d}] {value}"
        self.time += 1
        return entry

class CSVLogger(DataLogger):
    """CSV format logger."""
    
    def __init__(self, name, columns):
        super().__init__(name)
        self.columns = columns
        # Add header
        self.entries.append(",".join(columns))
    
    def format_entry(self, value):
        if isinstance(value, dict):
            return ",".join(str(value.get(c, "")) for c in self.columns)
        return str(value)

# Test loggers
simple = SimpleLogger("Basic")
timed = TimestampLogger("Timed")
csv = CSVLogger("CSV", ["temp", "pressure", "humidity"])

# Log some data
for i in range(3):
    simple.log(f"Reading {i}")
    timed.log(f"Sensor data {i}")
    csv.log({"temp": 25+i, "pressure": 1013+i, "humidity": 50+i})

print("Simple Logger:")
for e in simple.get_all():
    print(f"  {e}")

print("\nTimestamp Logger:")
for e in timed.get_all():
    print(f"  {e}")

print("\nCSV Logger:")
for e in csv.get_all():
    print(f"  {e}")

---
## Part 9: Additional Engineering Examples


Let's explore more advanced inheritance patterns used in real-world engineering applications.

### State Machine Pattern

**Figure 9.1: State Machine with Inheritance**

In [ ]:
# State Machine pattern using inheritance
from abc import ABC, abstractmethod

class State(ABC):
    """Abstract state."""
    @abstractmethod
    def handle(self, context):
        pass

class IdleState(State):
    def handle(self, context):
        print(f"{context.name}: Idle -> Ready")
        context.state = ReadyState()

class ReadyState(State):
    def handle(self, context):
        print(f"{context.name}: Ready -> Running")
        context.state = RunningState()

class RunningState(State):
    def handle(self, context):
        print(f"{context.name}: Running -> Complete")
        context.state = CompleteState()

class CompleteState(State):
    def handle(self, context):
        print(f"{context.name}: Complete -> Idle (reset)")
        context.state = IdleState()

class Machine:
    def __init__(self, name):
        self.name = name
        self.state = IdleState()
    
    def step(self):
        self.state.handle(self)
    
    def get_state(self):
        return self.state.__class__.__name__

# Test state machine
machine = Machine("CNC_001")
print(f"Initial state: {machine.get_state()}")

for _ in range(5):
    machine.step()
    print(f"Current state: {machine.get_state()}")

### Command Pattern

**Figure 9.2: Command Pattern**

In [ ]:
# Command pattern for robot control
from abc import ABC, abstractmethod

class Command(ABC):
    """Abstract command."""
    @abstractmethod
    def execute(self):
        pass
    
    @abstractmethod
    def undo(self):
        pass

class MoveCommand(Command):
    def __init__(self, robot, direction, distance):
        self.robot = robot
        self.direction = direction
        self.distance = distance
    
    def execute(self):
        self.robot.move(self.direction, self.distance)
    
    def undo(self):
        opposite = {'N': 'S', 'S': 'N', 'E': 'W', 'W': 'E'}
        self.robot.move(opposite[self.direction], self.distance)

class RotateCommand(Command):
    def __init__(self, robot, angle):
        self.robot = robot
        self.angle = angle
    
    def execute(self):
        self.robot.rotate(self.angle)
    
    def undo(self):
        self.robot.rotate(-self.angle)

class Robot:
    def __init__(self, name):
        self.name = name
        self.x = 0
        self.y = 0
        self.heading = 0
    
    def move(self, direction, distance):
        moves = {'N': (0, 1), 'S': (0, -1), 'E': (1, 0), 'W': (-1, 0)}
        dx, dy = moves[direction]
        self.x += dx * distance
        self.y += dy * distance
        print(f"{self.name}: Moved {direction} {distance}m -> ({self.x}, {self.y})")
    
    def rotate(self, angle):
        self.heading = (self.heading + angle) % 360
        print(f"{self.name}: Rotated {angle}° -> heading {self.heading}°")

# Test command pattern
robot = Robot("R2D2")
commands = [
    MoveCommand(robot, 'N', 5),
    MoveCommand(robot, 'E', 3),
    RotateCommand(robot, 90)
]

print("=== Executing commands ===")
for cmd in commands:
    cmd.execute()

print("\n=== Undoing commands ===")
for cmd in reversed(commands):
    cmd.undo()

### Observer Pattern

**Figure 9.3: Observer Pattern for Sensors**

In [ ]:
# Observer pattern for sensor monitoring
from abc import ABC, abstractmethod

class Observer(ABC):
    """Abstract observer."""
    @abstractmethod
    def update(self, subject):
        pass

class Subject:
    """Observable subject."""
    def __init__(self):
        self._observers = []
    
    def attach(self, observer):
        self._observers.append(observer)
    
    def detach(self, observer):
        self._observers.remove(observer)
    
    def notify(self):
        for observer in self._observers:
            observer.update(self)

class TemperatureSensor(Subject):
    def __init__(self, sensor_id):
        super().__init__()
        self.sensor_id = sensor_id
        self._temperature = 0
    
    @property
    def temperature(self):
        return self._temperature
    
    @temperature.setter
    def temperature(self, value):
        self._temperature = value
        self.notify()

class DisplayObserver(Observer):
    def __init__(self, name):
        self.name = name
    
    def update(self, subject):
        print(f"[{self.name}] {subject.sensor_id}: {subject.temperature}°C")

class AlarmObserver(Observer):
    def __init__(self, threshold):
        self.threshold = threshold
    
    def update(self, subject):
        if subject.temperature > self.threshold:
            print(f"⚠️ ALARM: {subject.sensor_id} exceeded {self.threshold}°C!")

# Test observer pattern
sensor = TemperatureSensor("TEMP_001")
sensor.attach(DisplayObserver("Display1"))
sensor.attach(DisplayObserver("Display2"))
sensor.attach(AlarmObserver(30))

print("=== Temperature readings ===")
for temp in [22, 25, 28, 32, 35]:
    print(f"\nSetting temperature to {temp}°C:")
    sensor.temperature = temp

### Factory Pattern

**Figure 9.4: Factory Pattern**

In [ ]:
# Factory pattern for creating sensors
from abc import ABC, abstractmethod

class Sensor(ABC):
    def __init__(self, sensor_id):
        self.sensor_id = sensor_id
        self.value = None
    
    @abstractmethod
    def read(self):
        pass
    
    @property
    @abstractmethod
    def unit(self):
        pass

class TemperatureSensor(Sensor):
    @property
    def unit(self):
        return "°C"
    
    def read(self):
        self.value = 25.0  # Simulated
        return self.value

class PressureSensor(Sensor):
    @property
    def unit(self):
        return "hPa"
    
    def read(self):
        self.value = 1013.25  # Simulated
        return self.value

class HumiditySensor(Sensor):
    @property
    def unit(self):
        return "%"
    
    def read(self):
        self.value = 65.0  # Simulated
        return self.value

class SensorFactory:
    """Factory for creating sensors."""
    _sensor_types = {
        'temperature': TemperatureSensor,
        'pressure': PressureSensor,
        'humidity': HumiditySensor
    }
    
    @classmethod
    def create(cls, sensor_type, sensor_id):
        if sensor_type not in cls._sensor_types:
            raise ValueError(f"Unknown sensor type: {sensor_type}")
        return cls._sensor_types[sensor_type](sensor_id)
    
    @classmethod
    def register(cls, name, sensor_class):
        cls._sensor_types[name] = sensor_class

# Use the factory
sensors = [
    SensorFactory.create('temperature', 'TEMP_001'),
    SensorFactory.create('pressure', 'PRES_001'),
    SensorFactory.create('humidity', 'HUMI_001')
]

print("=== Sensor readings ===")
for sensor in sensors:
    value = sensor.read()
    print(f"{sensor.sensor_id}: {value} {sensor.unit}")

---
## ❌ Common Mistakes to Avoid


These are the most frequent errors students make with inheritance. Study them before the exercises!

**Forgetting to call `super().__init__()`**

                    If a child class defines `__init__` without calling `super().__init__(...)`, the parent's initialization is skipped. Parent attributes won't exist, causing `AttributeError` later.

**Overriding a method but losing parent behavior**

                    Completely replacing a parent method instead of extending it. If you want the parent's logic PLUS extra steps, call `super().method_name()` inside your override.

**Deep inheritance hierarchies**

`A → B → C → D → E` — 5 levels deep makes code hard to follow and debug. Prefer shallow hierarchies (2–3 levels max) and use composition for additional behavior.

**Confusing `isinstance()` and `type()`**

`type(obj) == Parent` fails for child instances! Use `isinstance(obj, Parent)` which returns `True` for both parent and child types — respects polymorphism.

**Diamond problem with multiple inheritance**

                    If class D inherits from B and C, which both inherit from A, the MRO (Method Resolution Order) determines which method runs. Use `ClassName.mro()` to inspect the order and avoid surprises.

---
# 📝 Exercises


### Exercise 1: Basic Inheritance  (Easy)

Create a `Vehicle` base class with `brand` and `model` attributes. Create a `Car` subclass that adds `num_doors`.

**Expected Output:**
```
Toyota Camry
Toyota Camry (4 doors)
```

<details>
<summary>💡 Hints</summary>

- Use `class Car(Vehicle):` syntax
- Call `super().__init__(brand, model)`
- Override `describe()` in Car
</details>

In [ ]:
# ✏️ [EX1]
# Create Vehicle and Car classes
class Vehicle:
    # Your code here
    pass

class Car(Vehicle):
    # Your code here
    pass

# Test
vehicle = Vehicle("Toyota", "Camry")
car = Car("Toyota", "Camry", 4)
print(vehicle.describe())
print(car.describe())

### Exercise 2: Using super()  (Easy)

Create a `Person` class and an `Employee` subclass. Use `super()` to call the parent constructor.

**Expected Output:**
```
Person: Ali, 25 years
Employee: Ayşe, 30 years, ID: E001
```

<details>
<summary>💡 Hints</summary>

- In Employee.__init__: `super().__init__(name, age)`
- Add `self.employee_id` after super call
- Override __str__ to include ID
</details>

In [ ]:
# ✏️ [EX2]
# Use super() correctly
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def __str__(self):
        return f"Person: {self.name}, {self.age} years"

class Employee(Person):
    # Use super().__init__() here
    pass

# Test
person = Person("Ali", 25)
employee = Employee("Ayşe", 30, "E001")
print(person)
print(employee)

### Exercise 3: Method Overriding  (Easy)

Override the `speak()` method in `Dog` and `Cat` classes.

**Expected Output:**
```
Some sound
Buddy says Woof!
Whiskers says Meow!
```

<details>
<summary>💡 Hints</summary>

- Define `def speak(self):` in each subclass
- Return `f"{self.name} says Woof!"` for Dog
- Return `f"{self.name} says Meow!"` for Cat
</details>

In [ ]:
# ✏️ [EX3]
class Animal:
    def __init__(self, name):
        self.name = name
    
    def speak(self):
        return "Some sound"

class Dog(Animal):
    # Override speak()
    pass

class Cat(Animal):
    # Override speak()
    pass

# Test
animal = Animal("Generic")
dog = Dog("Buddy")
cat = Cat("Whiskers")

print(animal.speak())
print(dog.speak())
print(cat.speak())

### Exercise 4: isinstance() Check  (Easy)

Create a function that uses `isinstance()` to handle different shape types.

**Expected Output:**
```
Rectangle: 50 sq units
Circle: 78.54 sq units
Unknown shape
```

<details>
<summary>💡 Hints</summary>

- Use `if isinstance(shape, Rectangle):`
- Circle area: `3.14159 * shape.r ** 2`
- Return "Unknown shape" for non-Shape types
</details>

In [ ]:
# ✏️ [EX4]
class Shape:
    pass

class Rectangle(Shape):
    def __init__(self, w, h):
        self.w = w
        self.h = h

class Circle(Shape):
    def __init__(self, r):
        self.r = r

def describe_shape(shape):
    # Use isinstance() to check type
    # Return appropriate description
    pass

# Test
print(describe_shape(Rectangle(10, 5)))
print(describe_shape(Circle(5)))
print(describe_shape("not a shape"))

### Exercise 5: Extending Methods  (Easy)

Create a `LoggedCalculator` that extends `Calculator` by logging operations.

**Expected Output:**
```
15
[LOG] add(10, 5) = 15
15
```

<details>
<summary>💡 Hints</summary>

- Override add(): call `result = super().add(a, b)`
- Print the log message with result
- Return the result after logging
</details>

In [ ]:
# ✏️ [EX5]
class Calculator:
    def add(self, a, b):
        return a + b

class LoggedCalculator(Calculator):
    # Override add() to print log and call super()
    pass

# Test
calc = Calculator()
logged_calc = LoggedCalculator()

print(calc.add(10, 5))
print(logged_calc.add(10, 5))

### Exercise 6: Polymorphism  (Easy)

Write a `process_all()` function that calls `process()` on a list of objects polymorphically.

**Expected Output:**
```
Processing text: Hello
Processing number: 84
Processing list: 3 items
```

<details>
<summary>💡 Hints</summary>

- Loop: `for proc in processors:`
- Call `print(proc.process())`
- Same method name works on all types
</details>

In [ ]:
# ✏️ [EX6]
class TextProcessor:
    def __init__(self, text):
        self.text = text
    def process(self):
        return f"Processing text: {self.text}"

class NumberProcessor:
    def __init__(self, number):
        self.number = number
    def process(self):
        return f"Processing number: {self.number * 2}"

class ListProcessor:
    def __init__(self, items):
        self.items = items
    def process(self):
        return f"Processing list: {len(self.items)} items"

def process_all(processors):
    # Call process() on each item
    pass

# Test
items = [
    TextProcessor("Hello"),
    NumberProcessor(42),
    ListProcessor([1, 2, 3])
]
process_all(items)

### Exercise 7: Multiple Inheritance  (Medium)

Create a `FlyingFish` class that inherits from both `Flyable` and `Swimmable`.

**Expected Output:**
```
Nemo is flying
Nemo is swimming
```

<details>
<summary>💡 Hints</summary>

- Use `class FlyingFish(Flyable, Swimmable):`
- Define __init__ to set `self.name`
- Inherits both fly() and swim() methods
</details>

In [ ]:
# ✏️ [EX7]
class Flyable:
    def fly(self):
        return f"{self.name} is flying"

class Swimmable:
    def swim(self):
        return f"{self.name} is swimming"

class FlyingFish:  # Multiple inheritance
    # Your code here
    pass

# Test
fish = FlyingFish("Nemo")
print(fish.fly())
print(fish.swim())

### Exercise 8: Mixin Class  (Medium)

Create a `TimestampMixin` that adds timestamp capability to any class.

**Expected Output:**
```
Event created at timestamp: 1
Event created at timestamp: 2
```

<details>
<summary>💡 Hints</summary>

- Increment class counter: `TimestampMixin._counter += 1`
- Set `self.timestamp = TimestampMixin._counter`
- Class attributes shared across all instances
</details>

In [ ]:
# ✏️ [EX8]
class TimestampMixin:
    _counter = 0
    
    def set_timestamp(self):
        # Set self.timestamp to incrementing value
        pass
    
    def get_timestamp(self):
        return self.timestamp

class Event(TimestampMixin):
    def __init__(self, name):
        self.name = name
        self.set_timestamp()
    
    def __str__(self):
        return f"{self.name} created at timestamp: {self.get_timestamp()}"

# Test
e1 = Event("Event")
e2 = Event("Event")
print(e1)
print(e2)

### Exercise 9: Abstract Base Class  (Medium)

Create an abstract `DataSource` class with abstract `read()` and `write()` methods.

**Expected Output:**
```
Cannot instantiate abstract class
FileSource read: data.txt
MemorySource read: memory data
```

<details>
<summary>💡 Hints</summary>

- Use `@abstractmethod` decorator
- Subclasses must implement all abstract methods
- FileSource.read returns `f"FileSource read: {self.filename}"`
</details>

In [ ]:
# ✏️ [EX9]
from abc import ABC, abstractmethod

class DataSource(ABC):
    # Define abstract read() and write() methods
    pass

class FileSource(DataSource):
    def __init__(self, filename):
        self.filename = filename
    # Implement abstract methods

class MemorySource(DataSource):
    def __init__(self):
        self.data = "memory data"
    # Implement abstract methods

# Test
try:
    ds = DataSource()
except TypeError:
    print("Cannot instantiate abstract class")

file_src = FileSource("data.txt")
mem_src = MemorySource()
print(file_src.read())
print(mem_src.read())

### Exercise 10: Class Hierarchy  (Medium)

Create a 3-level hierarchy: `Shape` → `Polygon` → `Rectangle`.

**Expected Output:**
```
Rectangle: 4 sides, area=50
```

<details>
<summary>💡 Hints</summary>

- Rectangle calls `super().__init__(4)`
- Each level overrides describe()
- Rectangle.describe includes area calculation
</details>

In [ ]:
# ✏️ [EX10]
class Shape:
    def describe(self):
        return "Shape"

class Polygon(Shape):
    def __init__(self, sides):
        self.sides = sides
    # Override describe

class Rectangle(Polygon):
    def __init__(self, width, height):
        # Call Polygon's __init__ with 4 sides
        self.width = width
        self.height = height
    
    def area(self):
        return self.width * self.height
    # Override describe

# Test
rect = Rectangle(10, 5)
print(rect.describe())

### Exercise 11: Method Resolution Order  (Medium)

Predict and verify the MRO for a diamond inheritance pattern.

**Expected Output:**
```
MRO: D -> B -> C -> A -> object
D.greet(): Hello from B
```

<details>
<summary>💡 Hints</summary>

- Get MRO: `D.__mro__` or `D.mro()`
- Format: `[c.__name__ for c in D.__mro__]`
- D inherits from B first, so B.greet() is used
</details>

In [ ]:
# ✏️ [EX11]
class A:
    def greet(self):
        return "Hello from A"

class B(A):
    def greet(self):
        return "Hello from B"

class C(A):
    def greet(self):
        return "Hello from C"

class D(B, C):
    pass

# Print MRO and test greet()
# Your code here

### Exercise 12: Sensor Hierarchy  (Medium)

Create a sensor class hierarchy with `Sensor` base class and specific sensor types.

**Expected Output:**
```
TEMP_001: 25.5 °C
PRES_001: 1013.25 hPa
```

<details>
<summary>💡 Hints</summary>

- TemperatureSensor: `super().__init__(sensor_id, "°C")`
- PressureSensor: `super().__init__(sensor_id, "hPa")`
- Use parent's read() and display() methods
</details>

In [ ]:
# ✏️ [EX12]
class Sensor:
    def __init__(self, sensor_id, unit):
        self.sensor_id = sensor_id
        self.unit = unit
        self.value = None
    
    def read(self, value):
        self.value = value
    
    def display(self):
        return f"{self.sensor_id}: {self.value} {self.unit}"

class TemperatureSensor(Sensor):
    # Initialize with unit="°C"
    pass

class PressureSensor(Sensor):
    # Initialize with unit="hPa"
    pass

# Test
temp = TemperatureSensor("TEMP_001")
pres = PressureSensor("PRES_001")

temp.read(25.5)
pres.read(1013.25)

print(temp.display())
print(pres.display())

### Exercise 13: Plugin System  (Challenge)

Create an abstract `Plugin` class and implement multiple concrete plugins.

<details>
<summary>💡 Hints</summary>

- Use ABC and abstractmethod for the base class
- Each plugin should have name property and execute() method
- Create a PluginManager to run all plugins
</details>

In [ ]:
# ✏️ [EX13]
from abc import ABC, abstractmethod

class Plugin(ABC):
    # Abstract property: name
    # Abstract method: execute(data)
    pass

class UppercasePlugin(Plugin):
    # Converts data to uppercase
    pass

class ReversePlugin(Plugin):
    # Reverses the data string
    pass

class PluginManager:
    def __init__(self):
        self.plugins = []
    
    def register(self, plugin):
        self.plugins.append(plugin)
    
    def run_all(self, data):
        # Run each plugin and print results
        pass

# Test
manager = PluginManager()
manager.register(UppercasePlugin())
manager.register(ReversePlugin())
manager.run_all("hello world")

### Exercise 14: Controller System  (Challenge)

Create a control system with OnOff, P, and PI controllers.

<details>
<summary>💡 Hints</summary>

- Base Controller class with setpoint and compute() method
- OnOffController: output 100 if below setpoint, 0 if above
- PController: output = Kp * error
- PIController extends PController with integral term
</details>

In [ ]:
# ✏️ [EX14]
# Create Controller hierarchy
class Controller:
    def __init__(self, setpoint):
        self.setpoint = setpoint
    
    def compute(self, current):
        raise NotImplementedError

class OnOffController(Controller):
    pass

class PController(Controller):
    def __init__(self, setpoint, kp):
        super().__init__(setpoint)
        self.kp = kp

class PIController(PController):
    pass

# Test
onoff = OnOffController(25)
p = PController(25, kp=2.0)
pi = PIController(25, kp=2.0, ki=0.5)

print(f"OnOff (current=20): {onoff.compute(20)}")
print(f"P (current=20): {p.compute(20)}")
print(f"PI (current=20): {pi.compute(20)}")

### Exercise 15: Game Character System  (Challenge)

Create a game character system with different character types using inheritance.

<details>
<summary>💡 Hints</summary>

- Character base class with name, health, attack()
- Warrior: high damage, low speed
- Mage: magic attack, can heal
- Rogue: fast, critical hits
</details>

In [ ]:
# ✏️ [EX15]
# Game character system
class Character:
    def __init__(self, name, health):
        self.name = name
        self.health = health
    
    def attack(self):
        return f"{self.name} attacks for 10 damage"
    
    def take_damage(self, amount):
        self.health -= amount
        return f"{self.name} takes {amount} damage (health: {self.health})"

class Warrior(Character):
    # Override attack for higher damage
    pass

class Mage(Character):
    # Override attack and add heal()
    pass

class Rogue(Character):
    # Override attack with critical chance
    pass

# Test
warrior = Warrior("Conan", 100)
mage = Mage("Gandalf", 60)
rogue = Rogue("Shadow", 70)

print(warrior.attack())
print(mage.attack())
print(mage.heal())
print(rogue.attack())

### Exercise 🌉: Bridge Exercise: Sneak Peek at Week 12  (Preview)

**Next week: Modules & Packages!** Your sensor logger code is now 300+ lines in a single file. Finding anything is painful. What if you could split it into organized, reusable pieces?

**Expected Output:**
```
Current file: sensor_logger.py (327 lines!)

Contents:
  Lines 1-50:    Custom exceptions
  Lines 51-120:  Base Sensor class
  Lines 121-180: TemperatureSensor, HumiditySensor, PressureSensor
  Lines 181-260: SensorLogger class
  Lines 261-327: Main program logic

Finding the HumiditySensor class? Scroll, scroll, scroll... 😫
Next week: split into modules — import what you need!
```

<details>
<summary>💡 Hints</summary>

- A single 300+ line file is hard to navigate, test, and reuse
- What if another project needs just the Sensor class? Copy-paste the whole file?
- Next week: `from sensors import TemperatureSensor` — clean imports from organized modules
</details>

In [ ]:
# ✏️ [EXBridge]
# Bridge Exercise: Growing File Size Problem
# Everything crammed into one massive file!

file_structure = {
    "sensor_logger.py": {
        "total_lines": 327,
        "sections": [
            ("Custom exceptions", "1-50"),
            ("Base Sensor class", "51-120"),
            ("TemperatureSensor, HumiditySensor, PressureSensor", "121-180"),
            ("SensorLogger class", "181-260"),
            ("Main program logic", "261-327"),
        ]
    }
}

print(f"Current file: sensor_logger.py "
      f"({file_structure['sensor_logger.py']['total_lines']} lines!)\n")
print("Contents:")
for name, lines in file_structure["sensor_logger.py"]["sections"]:
    print(f"  Lines {lines:10s}: {name}")

print(f"\nFinding the HumiditySensor class? Scroll, scroll, scroll... 😫")
print("Next week: split into modules — import what you need!")

# What it COULD look like:
print("\n--- Better structure with modules ---")
ideal = [
    "sensors/",
    "  __init__.py",
    "  base.py          → Sensor class",
    "  temperature.py   → TemperatureSensor",
    "  humidity.py      → HumiditySensor",
    "  exceptions.py    → Custom exceptions",
    "logger.py          → SensorLogger",
    "main.py            → Program entry point",
]
for line in ideal:
    print(f"  {line}")

---
## 🔬 Case Study: Sensor Data Logger (Part 4 of 6)


Continuing from Week 10's OOP design, we now use **inheritance** to create specialized sensor types with custom validation, while reusing all shared behavior from the base class.

**Goal:** Create an abstract `Sensor` base class with shared logic, then build `TemperatureSensor`, `HumiditySensor`, and `PressureSensor` subclasses that each define their own validation rules.

**What's new:** Abstract base class with `ABC`, `super().__init__()`, method overriding for `validate()`, polymorphic `process_readings()`.

**Case Study 4 — Sensor Data Logger: Inheritance Hierarchy**

In [ ]:
# === CASE STUDY Part 4: Sensor Data Logger — Inheritance ===
# Specialized sensor types via inheritance

from abc import ABC, abstractmethod

class SensorError(Exception):
    pass

class InvalidReadingError(SensorError):
    def __init__(self, sensor_id, value, valid_range):
        super().__init__(f"{sensor_id}: {value} outside {valid_range}")
        self.sensor_id = sensor_id
        self.value = value

# --- Abstract Base Sensor ---
class Sensor(ABC):
    """Abstract base class for all sensors."""
    
    def __init__(self, sensor_id, unit, valid_range):
        self.sensor_id = sensor_id
        self.unit = unit
        self.valid_range = valid_range
        self._readings = []
    
    @property
    @abstractmethod
    def sensor_type(self):
        """Each subclass must define its type."""
        pass
    
    def validate(self, value):
        """Default validation — check range. Override for custom rules."""
        low, high = self.valid_range
        if not (low <= value <= high):
            raise InvalidReadingError(
                self.sensor_id, value, self.valid_range)
    
    def add_reading(self, timestamp, value):
        """Add reading with validation."""
        self.validate(value)
        self._readings.append((timestamp, value))
    
    def get_stats(self):
        if not self._readings:
            return {"min": None, "max": None, "avg": None, "count": 0}
        vals = [v for _, v in self._readings]
        return {"min": min(vals), "max": max(vals),
                "avg": sum(vals)/len(vals), "count": len(vals)}
    
    def __str__(self):
        s = self.get_stats()
        return (f"{self.sensor_type}({self.sensor_id}): "
                f"{s['count']} readings")

# --- Specialized Subclasses ---
class TemperatureSensor(Sensor):
    """Temperature sensor with °C validation."""
    
    def __init__(self, sensor_id):
        super().__init__(sensor_id, "°C", (-40, 80))
    
    @property
    def sensor_type(self):
        return "Temperature"
    
    def validate(self, value):
        """Extended: also warn on rapid changes."""
        super().validate(value)  # Range check first
        if self._readings:
            last_val = self._readings[-1][1]
            if abs(value - last_val) > 10:
                print(f"  ⚡ {self.sensor_id}: Rapid change "
                      f"detected ({last_val} → {value})")

class HumiditySensor(Sensor):
    """Humidity sensor with % validation."""
    
    def __init__(self, sensor_id):
        super().__init__(sensor_id, "%", (0, 100))
    
    @property
    def sensor_type(self):
        return "Humidity"

class PressureSensor(Sensor):
    """Pressure sensor with hPa validation."""
    
    def __init__(self, sensor_id):
        super().__init__(sensor_id, "hPa", (300, 1100))
    
    @property
    def sensor_type(self):
        return "Pressure"
    
    def validate(self, value):
        """Extended: pressure must be integer-like."""
        super().validate(value)
        if value != round(value, 1):
            print(f"  📏 {self.sensor_id}: Rounding to 1 decimal")

# --- Polymorphic Processing ---
def process_readings(sensors, data):
    """Process readings for ANY sensor type — polymorphism!"""
    errors = []
    for sid, ts, val_str in data:
        sensor = sensors.get(sid)
        if not sensor:
            errors.append(f"Unknown: {sid}")
            continue
        try:
            sensor.add_reading(ts, float(val_str))
            print(f"  ✅ {sid}: {val_str} {sensor.unit}")
        except (ValueError, SensorError) as e:
            errors.append(str(e))
            print(f"  ⚠️ {sid}: {e}")
    return errors

# --- Demo ---
print("🏗️ Creating specialized sensors...\n")
sensors = {
    "T01": TemperatureSensor("T01"),
    "H01": HumiditySensor("H01"),
    "P01": PressureSensor("P01"),
}

for s in sensors.values():
    print(f"  📡 {s} (range: {s.valid_range} {s.unit})")

print(f"\n📝 Processing readings...\n")
data = [
    ("T01", "08:00", "22.5"),
    ("T01", "08:05", "23.1"),
    ("T01", "08:10", "35.0"),   # Rapid change!
    ("H01", "08:00", "45.2"),
    ("H01", "08:05", "110.0"),  # Out of range!
    ("P01", "08:00", "1013.25"),
    ("P01", "08:05", "1012.8"),
]

errors = process_readings(sensors, data)

print(f"\n📊 Final Stats:")
for s in sensors.values():
    stats = s.get_stats()
    if stats["count"] > 0:
        print(f"  {s}: avg={stats['avg']:.1f} {s.unit}")

# Polymorphism proof
print(f"\n🔍 isinstance checks:")
print(f"  T01 is Sensor? {isinstance(sensors['T01'], Sensor)}")
print(f"  T01 is TemperatureSensor? "
      f"{isinstance(sensors['T01'], TemperatureSensor)}")

print(f"\n✅ One process_readings() handles ALL sensor types!")
print("🔜 Next week: Split into modules for clean organization!")

> 💡 **Note:** **What's next?** Our inheritance hierarchy is clean and extensible, but all 100+ lines are in a single file. In **Week 12**, we'll organize this into a proper Python package with modules for exceptions, sensors, and the logger.

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,12}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-12 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
_ORIGINAL_HASHES = {
    "ex1": "efc26c09e63c52a56122c445f625731d",
    "ex2": "dc6f3c0b0568684c5b598934ddc25a6a",
    "ex3": "3e8975403e5aafc5b343ccbd7499f682",
    "ex4": "886fd44e7c299ec2046cb1acb403e7b3",
    "ex5": "f42042a7218265a0a46a4d5b67921ed5",
    "ex6": "bbb0d3be04c6072ff05c9416f4b05f13",
    "ex7": "343640cebe45c11973e7becb29ce6d56",
    "ex8": "10b1857ed3fa6ddf038d2032df9ae537",
    "ex9": "50fedd3411cff6fa4ea08fc03b1174f3",
    "ex10": "dfe3469fd4c7e18e8040e0686058458d",
    "ex11": "7bf9a80d3265ca11d8264e91e61b8742",
    "ex12": "47ce86fc404a5acebe1b2cf26d644e72",
    "ex13": "14279625662597e5d035305491a0197a",
    "ex14": "3c818e3c45f0081fce9f7ec4a97e1c71",
    "ex15": "59e2440000916559b2056e6222387a50"
}

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request, hashlib
import time as _time, datetime as _dt

WEEK = "Week_11"
URL  = "https://script.google.com/macros/s/AKfycbyf1D3HGSAX4MoIhNlAuWlGrFyyvbM5MIv7ZsLxrVDlATUihrRGEAaibvIZYlCfd8Me/exec"

# ══════════════════════════════════════════════════════════
# SELF-HEALING TIMER — works even if Cell 1 was skipped
# ══════════════════════════════════════════════════════════
_TRACKER_FILE = os.path.join(
    '/content' if os.path.isdir('/content') else '/tmp',
    f'.cp2_session_{WEEK}.json'
)
IDLE_THRESHOLD = 300

def _sh_calc_active(heartbeats):
    """Calculate active seconds from a heartbeat list."""
    if len(heartbeats) < 2: return 0
    active = 0
    for i in range(1, len(heartbeats)):
        gap = heartbeats[i] - heartbeats[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _collect_timing():
    """
    Collect timing data from all available sources:
      1. In-memory variables (timer cell ran in this kernel)
      2. Persistent tracker file (previous sessions / kernel restarts)
      3. Minimal fallback (nothing available)
    """
    _info = {
        'active_time': 0, 'wall_time': 0, 'cells_run': 0,
        'heartbeat_count': 0, 'session_count': 0,
        'session_start': '', 'session_end': _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': 'none'
    }

    # ── Source 1: In-memory (timer cell ran this kernel) ──
    _has_memory = False
    try:
        _s = _SESSION_START        # noqa: F821
        _hb = list(_HEARTBEATS)    # noqa: F821
        _cr = _CELLS_RUN[0]        # noqa: F821
        _has_memory = True
        _info['active_time'] = _sh_calc_active(_hb)
        _info['wall_time'] = int(_time.time() - _s)
        _info['cells_run'] = _cr
        _info['heartbeat_count'] = len(_hb)
        _info['session_start'] = _SESSION_START_STR  # noqa: F821
        _info['source'] = 'memory'
    except NameError:
        pass

    # ── Source 2: Persistent file (adds previous sessions) ──
    try:
        with open(_TRACKER_FILE, 'r') as f:
            _t = json.load(f)
        sessions = _t.get('sessions', [])
        _info['session_count'] = len(sessions)

        if not _has_memory and sessions:
            # Timer cell was NOT run — reconstruct from file
            last = sessions[-1]
            hb = last.get('heartbeats', [])
            _info['active_time'] = _sh_calc_active(hb)
            _info['wall_time'] = int(hb[-1] - hb[0]) if len(hb) >= 2 else 0
            _info['cells_run'] = last.get('cells_run', len(hb))
            _info['heartbeat_count'] = len(hb)
            _info['session_start'] = last.get('start', '')
            _info['source'] = 'file-current'

        # Add previous sessions' active time
        if len(sessions) > 1:
            prev_active = 0
            prev_cells = 0
            prev_hb = 0
            cutoff = -1 if _has_memory else -1  # exclude current session
            for sess in sessions[:cutoff]:
                hb = sess.get('heartbeats', [])
                prev_active += _sh_calc_active(hb)
                prev_cells += sess.get('cells_run', len(hb))
                prev_hb += len(hb)
            _info['prev_active_time'] = prev_active
            _info['prev_cells_run'] = prev_cells
            _info['prev_heartbeats'] = prev_hb
            _info['total_active_time'] = _info['active_time'] + prev_active
            _info['total_cells_run'] = _info['cells_run'] + prev_cells
            if _info['source'] == 'none':
                _info['source'] = 'file-prev'

    except (FileNotFoundError, json.JSONDecodeError, KeyError, ValueError):
        pass

    # Ensure totals exist
    _info.setdefault('total_active_time', _info['active_time'])
    _info.setdefault('total_cells_run', _info['cells_run'])
    return _info

# ── Persist final heartbeat snapshot before collecting ──
try:
    _t = json.load(open(_TRACKER_FILE))
    if _t['sessions']:
        _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
        _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
        json.dump(_t, open(_TRACKER_FILE, 'w'))
except Exception:
    pass

_timing = _collect_timing()

# ── Display timing ──
if _timing['source'] == 'none':
    print("⚠️  Zamanlayıcı verisi bulunamadı — ilk hücreyi çalıştırmayı unutmayın.")
    print("    Gönderim yine de yapılacak (süre: 0 olarak kaydedilir).")
else:
    _a = _timing['active_time']
    _w = _timing['wall_time']
    print(f"⏱️  Bu oturum — Aktif: {_a//60}m {_a%60}s  |  Duvar: {_w//60}m {_w%60}s")
    print(f"🔢  Hücreler: {_timing['cells_run']}  |  Heartbeat: {_timing['heartbeat_count']}")
    if _timing['session_count'] > 1:
        _ta = _timing['total_active_time']
        print(f"📂  Toplam {_timing['session_count']} oturum — Toplam aktif: {_ta//60}m {_ta%60}s  |  Toplam hücre: {_timing['total_cells_run']}")
    if _timing['total_active_time'] < 120:
        print("⚠️  Toplam aktif süre < 2 dk — çalıştırdığınız hücre sayısı düşük olabilir.")

_time_on_page = _timing['total_active_time']

# ══════════════════════════════════════════════════════════
# STUDENT INFO CHECK
# ══════════════════════════════════════════════════════════
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın (öğrenci bilgileri).")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın — bazı alanlar boş.")

# ══════════════════════════════════════════════════════════
# EXERCISE ANSWER COLLECTION (improved with retry)
# ══════════════════════════════════════════════════════════
_EX_PATTERN = re.compile(r'#\s*✏️\s*\[EX(\d+)\]')

def _extract_exercise(src):
    """Try to extract exercise ID and code from a source string."""
    m = _EX_PATTERN.match(src)
    if not m: return None, None
    ex_id = 'ex' + m.group(1)
    clean = '\n'.join(src.split('\n')[1:]).strip()
    return ex_id, clean

_answers = {}

# ── Method 1: In[] list (most reliable for Run-All) ──
try:
    for _src in In:
        if not _src: continue
        _eid, _code = _extract_exercise(_src)
        if _eid:
            _answers[_eid] = {
                'code': _code,
                'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
            }
except NameError:
    pass

# ── Method 2: IPython history_manager (flush first!) ──
if not _answers:
    try:
        _ipy = get_ipython()
        # Force flush so Run-All history is fully written
        try: _ipy.history_manager.db.commit()
        except Exception: pass
        _time.sleep(0.5)  # small delay for DB sync
        for _sess, _line, _src in _ipy.history_manager.get_range(output=False):
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                }
    except Exception:
        pass

# ── Method 3: Read notebook file (VS Code / local Jupyter) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir('.') if f.endswith('.ipynb') and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), 'r', encoding='utf-8') as _f:
            _nb = json.load(_f)
        for _cell in _nb['cells']:
            if _cell['cell_type'] != 'code': continue
            _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                }

# ── Method 4: Colab notebook file at /content/ ──
if not _answers:
    try:
        _colab_candidates = [f for f in os.listdir('/content') if f.endswith('.ipynb') and WEEK in f]
        if _colab_candidates:
            _nb_path = os.path.join('/content', _colab_candidates[0])
            with open(_nb_path, 'r', encoding='utf-8') as _f:
                _nb = json.load(_f)
            for _cell in _nb['cells']:
                if _cell['cell_type'] != 'code': continue
                _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
                _eid, _code = _extract_exercise(_src)
                if _eid:
                    _answers[_eid] = {
                        'code': _code,
                        'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                    }
    except Exception:
        pass

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys(), key=lambda x: int(x[2:]) if x[2:].isdigit() else 999))}")

if not _answers:
    print("\n⚠️  Hiçbir exercise yanıtı bulunamadı!")
    print("Submit'ten ÖNCE tüm exercise hücrelerini çalıştırdığınızdan emin olun.")
    print("\n💡 İpucu: 'Run All' yerine exercise hücrelerini tek tek çalıştırıp")
    print("   ardından bu hücreyi çalıştırmayı deneyin.")
    raise SystemExit()

# ══════════════════════════════════════════════════════════
# SEND SUBMISSION
# ══════════════════════════════════════════════════════════
_data = json.dumps({
    'week': WEEK,
    'studentId': _sid,
    'studentName': _sname,
    'studentEmail': _semail,
    'classCode': _scode,
    'source': 'cp2-notebook',
    'timeOnPage': _time_on_page,
    'wallTime': _timing['wall_time'],
    'cellsRun': _timing['total_cells_run'],
    'sessionCount': _timing['session_count'],
    'sessionStart': _timing['session_start'],
    'sessionEnd': _timing['session_end'],
    'timerSource': _timing['source'],
    'answers': _answers
}).encode('utf-8')

print("📡 Gönderiliyor...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get('success'):
        print(f"\n✅ {_result['message']}")
        print('📧 Onay için e-postanızı kontrol edin.')
    else:
        print(f"\n❌ {_result.get('message', 'Gönderim başarısız')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  İstek gönderildi — onay için e-postanızı kontrol edin.")
    print(f"(E-posta gelmezse tekrar deneyin veya hocaya başvurun)")
